In [12]:
import pandas as pd
import numpy as np
import json

In [4]:
df = pd.read_csv('DailyTotalFemaleBirths.csv')
df

,Date,Births
0,1/1/1959,35
1,1/2/1959,32
2,1/3/1959,30
3,1/4/1959,31
4,1/5/1959,44
...,...,...
360,12/27/1959,37
361,12/28/1959,52
362,12/29/1959,48
363,12/30/1959,55


In [5]:
births = df['Births'].values
births_array = np.array(births)
bins = [np.percentile(births_array, 33), np.percentile(births_array, 66)]
def phan_loai_sinh(birth):
    if birth <= bins[0]:
        return 0  # Thấp
    elif birth <= bins[1]:
        return 1  # Trung bình
    else:
        return 2  # Cao

In [6]:
observations = [phan_loai_sinh(b) for b in births]
N = 3  # Số trạng thái ẩn
M = 3  # Số biểu tượng quan sát (nhóm)

In [8]:
# Khởi tạo tham số HMM
np.random.seed(42)
pi = np.array([1.0/N] * N)  # Phân phối ban đầu đồng đều
A = np.random.dirichlet(np.ones(N), N)  # Ma trận chuyển trạng thái
B = np.random.dirichlet(np.ones(M), N)  # Ma trận phát sinh

# Chuẩn hóa để đảm bảo xác suất hợp lệ
A = A / A.sum(axis=1, keepdims=True)
B = B / B.sum(axis=1, keepdims=True)

In [9]:
# Thuật toán Forward
def forward_algorithm(obs, A, B, pi):
    T = len(obs)
    alpha = np.zeros((T, N))
    # Khởi tạo
    for i in range(N):
        alpha[0, i] = pi[i] * B[i, obs[0]]
    # Đệ quy
    for t in range(1, T):
        for j in range(N):
            alpha[t, j] = sum(alpha[t-1, i] * A[i, j] for i in range(N)) * B[j, obs[t]]
    return alpha

# Thuật toán Backward
def backward_algorithm(obs, A, B):
    T = len(obs)
    beta = np.zeros((T, N))
    # Khởi tạo
    beta[-1, :] = 1.0
    # Đệ quy
    for t in range(T-2, -1, -1):
        for i in range(N):
            beta[t, i] = sum(A[i, j] * B[j, obs[t+1]] * beta[t+1, j] for j in range(N))
    return beta

In [10]:
# Thuật toán Baum-Welch
def baum_welch(obs, A, B, pi, max_iter=100, tol=1e-4):
    T = len(obs)
    for iteration in range(max_iter):
        # Bước E: Tính xác suất forward và backward
        alpha = forward_algorithm(obs, A, B, pi)
        beta = backward_algorithm(obs, A, B)
        
        # Tính xi và gamma
        xi = np.zeros((T-1, N, N))
        gamma = np.zeros((T, N))
        
        for t in range(T-1):
            denom = sum(alpha[t, i] * sum(A[i, j] * B[j, obs[t+1]] * beta[t+1, j]
                                        for j in range(N)) for i in range(N))
            for i in range(N):
                gamma[t, i] = sum(alpha[t, i] * beta[t, i] for i in range(N)) / sum(
                    alpha[t, k] * beta[t, k] for k in range(N))
                for j in range(N):
                    xi[t, i, j] = (alpha[t, i] * A[i, j] * B[j, obs[t+1]] * beta[t+1, j]) / denom
        
        # Tính gamma cho bước thời gian cuối
        gamma[T-1, :] = alpha[T-1, :] * beta[T-1, :] / sum(alpha[T-1, k] * beta[T-1, k] for k in range(N))
        
        # Bước M: Cập nhật tham số
        # Cập nhật pi
        new_pi = gamma[0, :]
        
        # Cập nhật A
        new_A = np.zeros((N, N))
        for i in range(N):
            for j in range(N):
                new_A[i, j] = sum(xi[t, i, j] for t in range(T-1)) / sum(gamma[t, i] for t in range(T-1))
        
        # Cập nhật B
        new_B = np.zeros((N, M))
        for j in range(N):
            for k in range(M):
                new_B[j, k] = sum(gamma[t, j] if obs[t] == k else 0 for t in range(T)) / sum(gamma[t, j] for t in range(T))
        
        # Chuẩn hóa để đảm bảo xác suất hợp lệ
        new_A = new_A / new_A.sum(axis=1, keepdims=True)
        new_B = new_B / new_B.sum(axis=1, keepdims=True)
        new_pi = new_pi / new_pi.sum()
        
        # Kiểm tra hội tụ
        if (np.max(np.abs(new_A - A)) < tol and
            np.max(np.abs(new_B - B)) < tol and
            np.max(np.abs(new_pi - pi)) < tol):
            print(f"Hội tụ sau {iteration+1} vòng lặp")
            break
        
        # Cập nhật tham số
        A, B, pi = new_A, new_B, new_pi
    
    return A, B, pi

In [11]:
# Thuật toán Viterbi (để giải mã trạng thái ẩn)
def viterbi(obs, A, B, pi):
    T = len(obs)
    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)
    states = np.zeros(T, dtype=int)
    
    # Khởi tạo
    for i in range(N):
        delta[0, i] = pi[i] * B[i, obs[0]]
    
    # Đệ quy
    for t in range(1, T):
        for j in range(N):
            max_val = 0
            max_idx = 0
            for i in range(N):
                val = delta[t-1, i] * A[i, j]
                if val > max_val:
                    max_val = val
                    max_idx = i
            delta[t, j] = max_val * B[j, obs[t]]
            psi[t, j] = max_idx
    
    # Kết thúc
    states[T-1] = np.argmax(delta[T-1, :])
    
    # Backtracking
    for t in range(T-2, -1, -1):
        states[t] = psi[t+1, states[t+1]]
    
    return states

In [13]:
# Hàm xuất mô hình sang tệp JSON
def export_model(A, B, pi, filename='hmm_model.json'):
    model = {
        'transition_matrix_A': A.tolist(),
        'emission_matrix_B': B.tolist(),
        'initial_distribution_pi': pi.tolist()
    }
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(model, f, indent=4, ensure_ascii=False)
    print(f"Mô hình đã được xuất ra tệp {filename}")

In [14]:
# Huấn luyện HMM
A, B, pi = baum_welch(observations, A, B, pi)

# Xuất mô hình
export_model(A, B, pi)

# Giải mã trạng thái ẩn
hidden_states = viterbi(observations, A, B, pi)

# In kết quả
print("Ma trận chuyển trạng thái (A):")
print(A)
print("\nMa trận phát sinh (B):")
print(B)
print("\nPhân phối trạng thái ban đầu (pi):")
print(pi)
print("\n10 trạng thái ẩn đầu tiên (0=Thấp, 1=Trung bình, 2=Cao):")
print(hidden_states[:10])

Hội tụ sau 3 vòng lặp
Mô hình đã được xuất ra tệp hmm_model.json
Ma trận chuyển trạng thái (A):
[[0.07886567 0.48554269 0.43559164]
 [0.58761425 0.16181761 0.25056814]
 [0.01445864 0.5186941  0.46684726]]

Ma trận phát sinh (B):
[[0.33219325 0.39533742 0.27246933]
 [0.33207463 0.39519625 0.27272912]
 [0.3320725  0.39519372 0.27273378]]

Phân phối trạng thái ban đầu (pi):
[0.33333333 0.33333333 0.33333333]

10 trạng thái ẩn đầu tiên (0=Thấp, 1=Trung bình, 2=Cao):
[2 1 0 1 0 1 0 1 0 1]
